In [1]:
import subprocess
import clingo

## Writing the Code

In [2]:
%%file test.lp

% Maximum number of states ("emergency brake")
#const n = 12.

e(a,b). e(b,c). e(c,d). e(d,e). e(e,f). e(f,a).

fr. % use frame rule, otherwise termination check fails 
% dr. % use double-recursive rule
rr. % use right-recursive rule
 
% Initialize transitive closure tc/2 from facts e/2:
tc(0, X,Y) :- e(X,Y).

% Frame rule: keep everything you had
tc(N1, X,Y) :- fr, next(N,N1),
	       tc(N, X,Y).

% % (dr) variant: double-recursive rule
% tc(N1, X,Y) :- dr, next(N,N1),
% 	       tc(N, X,Z),
% 	       tc(N, Z,Y).

% (rr) variant: right-recursive rule
tc(N1, X,Y) :- rr, next(N,N1),
	       e(X,Z),
	       tc(N, Z,Y).

% Which atoms have been newly derived in this round? 
new_tc(N1,X,Y) :- next(N,N1),
	       tc(N1,X,Y),
	       not tc(N,X,Y).

% Compute next state, if there's a need for it
next(0,1). 
next(N1,N2) :- next(N,N1), N2 = N1+1,
	       new_tc(N1, _, _), % if there's something new in N1 => create N2
	       N1 < n.        % this emergency brake is necessary (unfair eval/grounding strategy)

% What's the max state index used? 
max_state(N1) :- next(N,N1), not next(N1, N1+1).

%#show tc/3.
#show new_tc/3.
#show max_state/1.

Overwriting test.lp


## Show Content

In [3]:
!cat test.lp


% Maximum number of states ("emergency brake")
#const n = 12.

e(a,b). e(b,c). e(c,d). e(d,e). e(e,f). e(f,a).

fr. % use frame rule, otherwise termination check fails 
% dr. % use double-recursive rule
rr. % use right-recursive rule
 
% Initialize transitive closure tc/2 from facts e/2:
tc(0, X,Y) :- e(X,Y).

% Frame rule: keep everything you had
tc(N1, X,Y) :- fr, next(N,N1),
	       tc(N, X,Y).

% % (dr) variant: double-recursive rule
% tc(N1, X,Y) :- dr, next(N,N1),
% 	       tc(N, X,Z),
% 	       tc(N, Z,Y).

% (rr) variant: right-recursive rule
tc(N1, X,Y) :- rr, next(N,N1),
	       e(X,Z),
	       tc(N, Z,Y).

% Which atoms have been newly derived in this round? 
new_tc(N1,X,Y) :- next(N,N1),
	       tc(N1,X,Y),
	       not tc(N,X,Y).

% Compute next state, if there's a need for it
next(0,1). 
next(N1,N2) :- next(N,N1), N2 = N1+1,
	       new_tc(N1, _, _), % if there's something new in N1 => create N2
	       N1 < n.        % this emergency brake is necessary (unfair eval/groun

## Using Command Line

In [4]:
def run_bash(command):
    stdout = subprocess.getoutput(command)
    result=stdout.split("\n")[4].split(" ")
    return(result)

In [5]:
command='bash -c "clingo test.lp"'
result=run_bash(command)

In [6]:
result

['new_tc(1,f,b)',
 'new_tc(1,a,c)',
 'new_tc(1,b,d)',
 'new_tc(1,c,e)',
 'new_tc(1,d,f)',
 'new_tc(1,e,a)',
 'new_tc(2,e,b)',
 'new_tc(2,f,c)',
 'new_tc(2,a,d)',
 'new_tc(2,b,e)',
 'new_tc(2,c,f)',
 'new_tc(2,d,a)',
 'new_tc(3,d,b)',
 'new_tc(3,e,c)',
 'new_tc(3,f,d)',
 'new_tc(3,a,e)',
 'new_tc(3,b,f)',
 'new_tc(3,c,a)',
 'new_tc(4,c,b)',
 'new_tc(4,d,c)',
 'new_tc(4,e,d)',
 'new_tc(4,f,e)',
 'new_tc(4,a,f)',
 'new_tc(4,b,a)',
 'new_tc(5,b,b)',
 'new_tc(5,c,c)',
 'new_tc(5,d,d)',
 'new_tc(5,e,e)',
 'new_tc(5,f,f)',
 'new_tc(5,a,a)',
 'max_state(6)']

## Using Clingo Python Pkg

In [7]:
result=[]
global result

def on_model(model):
    print("Found solution:", model)
    for answer_set in str(model).split(" "):
        result.append(answer_set)

with open('test.lp') as f:
    asp_code = f.read()

ctl = clingo.Control()
ctl.add("base", [], asp_code)
parts = []
parts.append(("base", []))
ctl.ground(parts)
ret=ctl.solve(on_model=on_model)

Found solution: new_tc(1,f,b) new_tc(1,a,c) new_tc(1,b,d) new_tc(1,c,e) new_tc(1,d,f) new_tc(1,e,a) new_tc(2,e,b) new_tc(2,f,c) new_tc(2,a,d) new_tc(2,b,e) new_tc(2,c,f) new_tc(2,d,a) new_tc(3,d,b) new_tc(3,e,c) new_tc(3,f,d) new_tc(3,a,e) new_tc(3,b,f) new_tc(3,c,a) new_tc(4,c,b) new_tc(4,d,c) new_tc(4,e,d) new_tc(4,f,e) new_tc(4,a,f) new_tc(4,b,a) new_tc(5,b,b) new_tc(5,c,c) new_tc(5,d,d) new_tc(5,e,e) new_tc(5,f,f) new_tc(5,a,a) max_state(6)


In [8]:
result

['new_tc(1,f,b)',
 'new_tc(1,a,c)',
 'new_tc(1,b,d)',
 'new_tc(1,c,e)',
 'new_tc(1,d,f)',
 'new_tc(1,e,a)',
 'new_tc(2,e,b)',
 'new_tc(2,f,c)',
 'new_tc(2,a,d)',
 'new_tc(2,b,e)',
 'new_tc(2,c,f)',
 'new_tc(2,d,a)',
 'new_tc(3,d,b)',
 'new_tc(3,e,c)',
 'new_tc(3,f,d)',
 'new_tc(3,a,e)',
 'new_tc(3,b,f)',
 'new_tc(3,c,a)',
 'new_tc(4,c,b)',
 'new_tc(4,d,c)',
 'new_tc(4,e,d)',
 'new_tc(4,f,e)',
 'new_tc(4,a,f)',
 'new_tc(4,b,a)',
 'new_tc(5,b,b)',
 'new_tc(5,c,c)',
 'new_tc(5,d,d)',
 'new_tc(5,e,e)',
 'new_tc(5,f,f)',
 'new_tc(5,a,a)',
 'max_state(6)']